#함수

가독성(readability)을 높이기 위해 함수(function)를 정의해서 사용할 수 있다.

In [0]:
def my_func(arg1, arg2):
  arg_sum = arg1 + arg2
  return arg_sum

In [0]:
my_func(1,2)

3

#data loading

In [0]:
from google.colab import drive
from datetime import datetime

import os
import pandas as pd

In [0]:
# 구글 드라이브 접근 권한 획득
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# 구글 드라이브 접근 확인하기
!ls -lt "/content/gdrive/My Drive/ML_Datasets"

total 3388783
-rw------- 1 root root 1441757828 Jul 25 09:00  loan_seg_df_3.pkl
-rw------- 1 root root  836188751 Jul 23 08:49  loan_seg_df_2.pkl
-rw------- 1 root root 1189395649 Jul 23 07:38  loan.csv
-r-------- 2 root root      23594 Jul 13 09:01  LCDataDictionary.xlsx
-rw------- 1 root root    2745852 Jul 12 08:50 'Ecommerce Purchases.csv'


In [0]:
# 피클파일 읽기
folder_path = r'/content/gdrive/My Drive/ML_Datasets'
pkl_fname = 'loan_seg_df_3.pkl'
pkl_fpath = folder_path + os.sep + pkl_fname
print('파일경로: ', pkl_fpath)
loan_seg_df = pd.read_pickle(pkl_fpath)
print('파일형태: ', loan_seg_df.shape)
loan_seg_df.head(2)

파일경로:  /content/gdrive/My Drive/ML_Datasets/loan_seg_df_3.pkl
파일형태:  (1962274, 95)


id member_id  loan_amnt  ...  loan_status_curr_fg  issue_yymmdd issue_yymm
186                    4500  ...                False    2018-12-01     201812
269                   20000  ...                False    2018-12-01     201812

[2 rows x 95 columns]

In [0]:
# 인덱스를 새롭게 생성합니다.
loan_seg_df.reset_index(drop=True, inplace=True)
loan_seg_df.head(2)

id member_id  loan_amnt  ...  loan_status_curr_fg  issue_yymmdd issue_yymm
0                    4500  ...                False    2018-12-01     201812
1                   20000  ...                False    2018-12-01     201812

[2 rows x 95 columns]

In [0]:
#처리 속도를 위해 50만건만 처리해보자.
loan_half_m_df = loan_seg_df.head(500000).copy()
loan_half_m_df.shape

(500000, 95)

#new column generation

iloc: index location

새로운 변수를 추가할 때 brute-force하게 무식하게 for문을 돌릴 수 있다.

##apply

for문으로 iloc[4]를 찾는 것과, iloc[5]를 찾는 것은 별개의 일입니다. 처음부터 다시 찾을 것입니다.

하지만 apply는? 4 다음이 5라는 걸 알고 있기 때문에 처음부터 다시 찾지 않고 처리합니다.

In [0]:
#apply 함수 생성
def apply_func_one_one(open_acc):
  new_col = open_acc*10
  return new_col

In [0]:
loan_half_m_df['new_col'] = loan_half_m_df.apply(lambda x: apply_func_one_one(x['open_acc']), axis=1)
#axis=1인 경우 record 단위로 접근하여 return해줍니다.
loan_half_m_df[['open_acc', 'new_col']].head(2)

<built-in method total_seconds of datetime.timedelta object at 0x7fb5cd03bf58>


open_acc  new_col
0      12.0    120.0
1      10.0    100.0

너무 느립니다. 좀더 단순화해봅시다.

In [0]:
#필요한 column에만 접근하는 식
loan_half_m_df['new_col'] = loan_half_m_df['open_acc'].apply(lambda x: apply_func_one_one(x))
loan_half_m_df[['open_acc', 'new_col']].head(2)

open_acc  new_col
0      12.0    120.0
1      10.0    100.0

In [0]:
#간단한 수식이므로, 함수 선언 없이 해보면 더 빠르다.
loan_half_m_df['new_col'] = loan_half_m_df['open_acc'] * 10
loan_half_m_df[['open_acc','new_col']].head(2)

open_acc  new_col
0      12.0    120.0
1      10.0    100.0

return Series

record도 series기 때문에 series를 호출해야 합니다. 때문에 연산이 느립니다.

return tuple

series를 호출하지 않으므로 가장 빠른 방법입니다.

return row

row 전체를 호출하고 update하기 때문에 가장 오래 걸리는 방법입니다.

##vectorization

In [0]:
def vec_func_one_output(open_acc, total_acc):
  new_col = open_acc + total_acc
  return new_col

In [0]:
#vector로 호출해봅시다. 직접 함수를 쓰면 됩니다.
loan_half_m_df['new_col'] = vec_func_one_output(loan_half_m_df['open_acc'], loan_half_m_df['total_acc'])
loan_half_m_df[['open_acc','total_acc', 'new_col']].head(2)

open_acc  total_acc  new_col
0      12.0       25.0     37.0
1      10.0       16.0     26.0

vector는 빠르지만, 가장 간단한 수준의 연산만 가능합니다. numpy와 함께 활용해보면 좋습니다.

암묵적 vector 연산이란, 시스템에서 암묵적으로 이 변수가 vector로 들어온다고 가정하는 것을 말합니다.

vector로 불가능하거나 귀찮은 것을 apply로 해보자.(scalar 연산)

In [0]:
loan_half_m_df['earliest_cr_line_dtim'] = loan_half_m_df['earliest_cr_line'].apply(lambda x: datetime.strptime(x, '%b-%Y'))
loan_half_m_df['earliest_cr_line_yy'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.year)
loan_half_m_df['earliest_cr_line_mm'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.month)
loan_half_m_df[['earliest_cr_line','earliest_cr_line_yy', 'earliest_cr_line_mm']].head(2)

earliest_cr_line  earliest_cr_line_yy  earliest_cr_line_mm
0         Dec-2003                 2003                   12
1         Apr-1995                 1995                    4

In [0]:
loan_half_m_df['earliest_cr_line_dtim'] = loan_half_m_df['earliest_cr_line'].apply(lambda x: pd.to_datetime(x, format='%b-%Y'))
#비슷하지만, pandas 모듈과 format을 사용했다.
loan_half_m_df['earliest_cr_line_yy'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.year)
loan_half_m_df['earliest_cr_line_mm'] = loan_half_m_df['earliest_cr_line_dtim'].apply(lambda x: x.month)
loan_half_m_df[['earliest_cr_line','earliest_cr_line_yy', 'earliest_cr_line_mm']].head(2)

earliest_cr_line  earliest_cr_line_yy  earliest_cr_line_mm
0         Dec-2003                 2003                   12
1         Apr-1995                 1995                    4

#merge

뭔가 data table을 합칠 때 column 쪽으로 붙일 수도 있고, record 쪽으로 붙일 수도 있다.

merge, join, concat 등으로 data를 병합할 수 있다.

##index merging

In [0]:
#실습 데이터를 만들어봅시다.
import pandas as pd
df_1 = pd.DataFrame(
        data = {
            'col1': [1,2,3],
            'col2': ['steve', 'bucky', 'tony'],
        }, index=[1,2,3])
df_1

col1   col2
1     1  steve
2     2  bucky
3     3   tony

In [0]:
df_2 = pd.DataFrame(
    data={
      'col3': [55, 33, 11],
      'col2': ['steve', 'bucky', 'tony'],
    },
    index=[5,3,1]
)
df_2

col3   col2
5    55  steve
3    33  bucky
1    11   tony

In [0]:
# inner
merged_df = df_1.merge(df_2, left_index=True, right_index=True, how='inner', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y _merge
1     1  steve    11   tony   both
3     3   tony    33  bucky   both

In [0]:
# outer
merged_df = df_1.merge(df_2, left_index=True, right_index=True, how='outer', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y      _merge
1   1.0  steve  11.0   tony        both
2   2.0  bucky   NaN    NaN   left_only
3   3.0   tony  33.0  bucky        both
5   NaN    NaN  55.0  steve  right_only

In [0]:
# left
merged_df = df_1.merge(df_2, left_index=True, right_index=True, how='left', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y     _merge
1     1  steve  11.0   tony       both
2     2  bucky   NaN    NaN  left_only
3     3   tony  33.0  bucky       both

In [0]:
# right
merged_df = df_1.merge(df_2, left_index=True, right_index=True, how='right', sort=True, indicator=True)
merged_df

col1 col2_x  col3 col2_y      _merge
1   1.0  steve    11   tony        both
3   3.0   tony    33  bucky        both
5   NaN    NaN    55  steve  right_only

merge할 때 중복 처리에 신경 써야 합니다.

##validate

In [0]:
# index 중복이 있는 경우
df_3 = pd.DataFrame(
    data={
      'col3': [55, 33, 33, 11],
      'col2': ['steve', 'bucky', 'bucky', 'tony'],
    },
    index=[5,3,3,1]
)
df_3

col3   col2
5    55  steve
3    33  bucky
3    33  bucky
1    11   tony

In [0]:
# one_to_one
merged_df = df_1.merge(df_3, left_index=True, right_index=True, how='inner', sort=True, indicator=True, validate='one_to_one')
merged_df

MergeError: ignored

중복은 처리하지 말라고 했는데 중복이 있다고 오류

##suffixes

suffix를 이용해 내가 원하는 입맛대로 처리하도록 설정할 수 있다.

##column merging

#practice

솔루션1은 python, 솔루션2는 sql에 가깝습니다.

In [0]:
app_df = pd.DataFrame(
      data={
          'id':['steve', 'bucky', 'tony', 'natasha'],
          'apply_loan_amt':[2000,2500,3000,3000]
      })
app_df

id  apply_loan_amt
0    steve            2000
1    bucky            2500
2     tony            3000
3  natasha            3000

In [0]:
income_df = pd.DataFrame(
        data={
            'id':['steve', 'steve', 'natasha'],
            'application_income': [60000, 50000, 80000],
            'certified_income': [55000, 45000, 75000]
        })
income_df

id  application_income  certified_income
0    steve               60000             55000
1    steve               50000             45000
2  natasha               80000             75000

신청고객 정보에 소득을 병합하세요.
신청서소득정보(컬럼)와 증명소득정보(컬럼) 중, 작은 값 최종소득정보(컬럼)으로 생성하세요.
동일 신청고객에 대해서 복수개의 최종소득정보 레코드가 존재하는 경우, 그 값이 큰 레코드를 사용하세요.
병합 시, 소득정보가 병합되지 않는 경우, 0을 입력하세요.

##solution 1

In [0]:
# 두 소득을 비교해서 낮은 소득을 최종적으로 표시하는 컬럼을 추가합니다. apply를 씁시다.
income_df['final_income'] = income_df[['application_income','certified_income']].apply(lambda x:min(x),axis=1)
income_df

id  application_income  certified_income  final_income
0    steve               60000             55000         55000
1    steve               50000             45000         45000
2  natasha               80000             75000         75000

In [0]:
# 어떤 record는 중복된 것 같습니다. 지우기 위해 일단 정렬을 먼저 해봅니다.
income_df.sort_values(by=['final_income'], axis=0, inplace=True, ascending=True)

id  application_income  certified_income  final_income
1    steve               50000             45000         45000
0    steve               60000             55000         55000
2  natasha               80000             75000         75000

In [0]:
# 중복되는 record를 drop해줍니다.
nodup_col=['id']
nodup_income_df = income_df.drop_duplicates(subset=nodup_col, inplace=False, keep='first')
nodup_income_df

id  application_income  certified_income  final_income
0    steve               60000             55000         55000
2  natasha               80000             75000         75000

In [0]:
#원하는 데이터를 위해 merge하는데, 왼쪽 데이터가 남아야할 것 같습니다. 없는 값은 0으로 설정합니다.
merged_df = app_df.merge(nodup_income_df[['id', 'final_income']], on=['id'], how='left').fillna(0)
merged_df

id  apply_loan_amt  final_income
0    steve            2000       55000.0
1    bucky            2500           0.0
2     tony            3000           0.0
3  natasha            3000       75000.0

##solution 2